# Day 18: RAM Run

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def read_data(is_test: bool = False, plot: bool = False):
    if is_test:
        with open('data/2024-18-example.txt', 'r') as f:
            data = f.readlines()
        n_bytes = 12
        memory_space = np.zeros((7, 7))
    else:
        with open('data/2024-18.txt', 'r') as f:
            data = f.readlines()
        n_bytes = 1024
        memory_space = np.zeros((71, 71))
    
    data = [x.strip() for x in data]
    bytes_pos = np.array([list(map(int, x.split(','))) for x in data])

    return memory_space, bytes_pos, n_bytes


memory_space, bytes_pos, n_bytes = read_data(plot=True)
memory_space[bytes_pos[:n_bytes, 0], bytes_pos[:n_bytes, 1]] = 1

plt.imshow(memory_space, cmap='gray')
plt.title('Memory Space')
plt.axis('off')
plt.show()

## Part 1:

In [ ]:
G = nx.Graph()
for i, j in zip(*np.where(memory_space == 0)):
    source = i * memory_space.shape[0] + j
    
    # add path to right node
    if i < memory_space.shape[0] - 1:
        if memory_space[i+1, j] == 0:
            target = (i + 1) * memory_space.shape[0] + j
            G.add_edge(source, target)
    # add path to left node
    if j < memory_space.shape[1] - 1:
        if memory_space[i, j+1] == 0:
            target = i * memory_space.shape[0] + j + 1
            G.add_edge(source, target)

start_pos = 0
end_pos = memory_space.shape[0] * memory_space.shape[1] - 1

shortest_path = nx.shortest_path(G, source=start_pos, target=end_pos)
print('Part 1:', len(shortest_path) - 1)

## Part 2

In [ ]:
for i in range(n_bytes, bytes_pos.shape[0]):
    # remove next corrupted byte from graph
    next_byte = bytes_pos[i]
    node_name = next_byte[0] * memory_space.shape[0] + next_byte[1]
    
    if G.has_node(node_name):
        G.remove_node(node_name)
    
    # check if path still exists
    if not nx.has_path(G, start_pos, end_pos):
        print('Part 2:', ','.join(next_byte.astype(str)))
        break